# Tensor Puzzles
See https://github.com/srush/Tensor-Puzzles. The idea of these is that all we have access to for creating arrays is `arange` and `where`, plus arithmetic/comparison, and have to use broadcasting and other tricks to recreate a bunch of numpy functions. We're supposed to do all of these in one-liners.

In [1]:
import numpy as np

In [2]:
# Puzzle 1: ones.
# Explanation: The np.arange(n) is just used to force broadcasting to an array of shape (n,).
# Then 0 always evaluates to false, so we pick the second argument.
def ones(n):
    return np.where(0, np.arange(n), 1)
ones(10)

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [3]:
# Puzzle 2: sum. 
# Explanation: 
# We're taking the dot-product of the vector with (1,1,1,1,1...,1), which gives the sum.
def sum(n):
    return n @ ones(n.shape[0])
assert sum(ones(10)) == sum(np.array([1, 2, 3, 4]))
assert sum(ones(10)) == 10
sum(np.arange(100))

4950

In [4]:
# Puzzle 3: outer
# Explanation: Suppose a, b have shapes (i,) and (j,)
# a[:, None] has shape (i, 1).
# So when we multiply an (i,1) and a (j,), it gets broadcast to an array with shape (i, j).
def outer(a, b):
    return a[:, None] * b
outer(np.arange(3), np.arange(4))

array([[0, 0, 0, 0],
       [0, 1, 2, 3],
       [0, 2, 4, 6]])

In [5]:
# Puzzle 4: diag
# Explanation: This uses advanced indexing - it's the multidimensional index array case for integer indexing
# If we have:
# - An ndarray M, with shape (x_1, ..., x_n)
# - n ndarrays K_i, of the same shape (or broadcastable to the same shape)
# - Then M[K_1, ..., K_n] has the same shape as the K_i's, and each element is the corresponding indexed element of M
#
# So M[np.arange(N), np.arange(N)] has shape (N,), where element i is M[i][i]. Which is the diagonal index.
def diag(M):
    return M[np.arange(M.shape[0]), np.arange(M.shape[0])]
M = np.array([np.arange(10) for _ in range(10)])
diag(M)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [6]:
# Puzzle 5: eye
# Explanation: 
# LHS of the boolean condition is an array of shape (n, 1)
# RHS of the boolean condition is an array of shape (n,)
# So (LHS == RHS)_{ij} = (LHS_i == RHS_j), which gives us the identity matrix. 
def eye(N):
    return np.where(np.arange(N)[:, None] == np.arange(N), 1, 0)
eye(10)

array([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1]])

In [7]:
# Puzzle 6: triu
# Similar to above, we have:
# (LHR == RHS)_{ij} = (LHS_i <= RHS_j), which gives us the upper triangular matrix.
def triu(N):
    return np.where(np.arange(N)[:, None] <= np.arange(N), 1, 0)
triu(10)

array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [0, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [0, 0, 1, 1, 1, 1, 1, 1, 1, 1],
       [0, 0, 0, 1, 1, 1, 1, 1, 1, 1],
       [0, 0, 0, 0, 1, 1, 1, 1, 1, 1],
       [0, 0, 0, 0, 0, 1, 1, 1, 1, 1],
       [0, 0, 0, 0, 0, 0, 1, 1, 1, 1],
       [0, 0, 0, 0, 0, 0, 0, 1, 1, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1]])

In [8]:
# Puzzle 7: cumsum.
# This works, but feels inefficient - I'm doing a pairwise multiplication and a matrix*vector, which are both O(n^2).
# 
# Explanation: 
# (triu(N).T)_{ij} = (1 if i >= j else 0). I.e. lower triangular matrix.
# x is of shape (n,), so it's going to get broadcast out to shape (n,n) when we multiply. So
# (triu(N).T * x)_{ij} = (x_j if i >= j else 0).
#
# Then taking the matrix product of this and the ones vector:
# (output_i) = \sum_j (x_j if i >= j else 0), which is the cumulative sum.
def cumsum(x):
    return triu(x.shape[0]).T * x @ ones(x.shape[0])
cumsum(np.arange(10))

array([ 0,  1,  3,  6, 10, 15, 21, 28, 36, 45])

In [9]:
# Puzzle 8: diff
# Explanation: diff is a linear function of the input.
# I'm multiplying by the matrix which has 1s down the diagonal, and -1s on the "off-diagonal".
# So X_{ij} = 1 if i == j, else -1 if i == j + 1, else 0.
def diff(m):
    N = m.shape[0]; return (eye(N) - (np.arange(N)[:, None] - np.arange(N) == 1)) @ m
diff(np.arange(3,10))

array([3, 1, 1, 1, 1, 1, 1])

In [10]:
# Puzzle 9: vstack
# Explanation:
# X is the matrix [[1,1], [0,0]], so 1-X is the matrix [[0,0], [1,1]].
# Then pairwise multiplying the first by m (using broadcasting in first axis) is [[m_0, m_1], [0,0]], and similarly for n
# So adding these gives the vstack result.
def vstack(m, n): 
    X = (np.arange(2) == 0)[:, None]; return m * X + n * (1-X)
vstack(np.arange(3, 10), np.arange(7))

array([[3, 4, 5, 6, 7, 8, 9],
       [0, 1, 2, 3, 4, 5, 6]])

In [11]:
# Puzzle 10: roll
# Explanation: roll is a linear function of the input, so I'm just expressing this with the linear map.
# X is the matrix e.g. [[0, 1, 0], [0, 0, 1], [1, 0, 0]], so "off-by-one" from an identity matrix.
def roll(x): 
    N = x.shape[0]; X = (np.arange(N)[:, None] - np.arange(N)) % N == N-1; return X @ x
roll(np.arange(1, 6))

array([2, 3, 4, 5, 1])

In [12]:
# Puzzle 11: flip
# Very similar principle to above - this is a linear function. 
def flip(x): 
    N = x.shape[0]; X = (np.arange(N)[:, None] + np.arange(N)) == N-1; return X @ x
flip(np.arange(1, 6))

array([5, 4, 3, 2, 1])

In [13]:
# Puzzle 12: compress 
# Explanation: I don't understand what's different about this to Boolean indexing. Maybe this is cheating?
# https://stackoverflow.com/questions/44487889/why-is-np-compress-faster-than-boolean-indexing suggests they are equivalent.
# But np.compress is a little faster (it's more specific, which skips some of the numpy foo).
def compress(b, v):
    return v[b]
compress(np.array([False, True, False, True]), np.array([1,2,3,4]))

array([2, 4])

In [14]:
# Puzzle 13: pad_to. 
# Explanation: We're going to use integer indexing. To do this, I need a way to pick 0 if I'm padding.
# We'll use vstack for this, just as a way to get an array with some 0s in.
#
# Then:
# A is going to be of length M, len(v) 0s followed by M - len(v) 1s. We can do this with a boolean condition, and then add 0 to cast to an int.
# B is going to be 0, 1, 2, ..., len(v)-1, and then the later indices don't matter but must be less than len(v). I took a modulus.
# X is vstack(v, 0).
#
# Then X[A, B] consists of min(M, len(v)) elements from v, followed by indices from the bottom row, which are always zero.
#
# I'm using the fact that my vstack works with scalars too - if not I'd need to pass in a 0 vector of the right length.
def pad_to(v, M):
    X = vstack(v, 0); N = len(v); A = (np.arange(M) >= N) + 0; B = np.arange(M) % N; return X[A, B]
print(pad_to(np.arange(5), 8))
print(pad_to(np.arange(5), 4))

[0 1 2 3 4 0 0 0]
[0 1 2 3]


In [15]:
# Puzzle 14: sequence_mask. This is a bit like pad_to but takes in an array of lengths instead
# I found the definition of this one a bit confusing - it doesn't seem to match up with Tensorflow's
#
# Explanation:
# X = (np.arange(N) < length[:, None]) creates an ndarray of shape (M, N) 
# X_{ij} is True iff j < length[i]
# So this then acts as the mask that we want, if we do pointwise multiplication.
def sequence_mask(values, length):
    (M, N) = values.shape; return (np.arange(N) < length[:, None]) * values
    
sequence_mask(np.ones((4, 3), dtype=int), np.array([2, 1, 1, 4]))

array([[1, 1, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 1, 1]])

In [16]:
# Puzzle 15: bincount. The interface is a little different from Numpy's - we're given an integer j to populate.
# Explanation: The matrix on the LHS is basically an indicator function. If it's X, then 
# X_{ij} == (v[j] == i)
#
# Then taking the matrix product with a vector ones sums up over all j, so counts the number of indices equal to i.
def bincount(v, j):
    return (v == np.arange(j)[:, None]) @ ones(v.shape[0])
    
bincount(np.array([0,5,3,2,1,1,1,1,1,1]), 7)

array([1, 6, 1, 1, 0, 1, 0])

In [17]:
# Puzzle 16: scatter_add
# Target: The output should have o[i] equal to the sum of the elements in values, at the indices where the link equals i
#
# Explanation: Very similar to above. The matrix is an indicator function, but of the link rather than the values.
# So X_{ij} == (link[j] == i)
# Then we take the matrix product with values gives us \sum_{j with link[j] == i} values[j], which is the definition of scatter_add
def scatter_add(values, link, j):
    return (link == np.arange(j)[:, None]) @ values
scatter_add(np.array([5, 1, 7, 2, 3, 2, 1, 3]), np.array([0, 0, 1, 0, 2, 2, 3, 3]), 4)

array([8, 7, 5, 4])

In [18]:
# Puzzle 17: flatten
# Explanation: Using numerical indices. We're indexing with a pair of ndarrays, so the output is equal to the shape of those arrays.
# It's a bit like if you were to loop over a 2D array in C, but with one loop.
def flatten(v): 
    (M, N) = v.shape; return v[np.arange(M * N) // N, np.arange(M*N) % N]
flatten(np.arange(30).reshape(6, 5))

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29])

In [19]:
# Puzzle 18: linspace
# Explanation: Apply an affine transformation to [0, 1, ..., n-1] 
def linspace(i, j, n):
    return np.arange(n) * (j-i)/(n-1) + i
linspace(3, 10, 10)

array([ 3.        ,  3.77777778,  4.55555556,  5.33333333,  6.11111111,
        6.88888889,  7.66666667,  8.44444444,  9.22222222, 10.        ])

In [20]:
# Puzzle 19: heaviside
# Explanation: This is just a single np.where clause. If a is 0, pick b, otherwise take (a>0).
def heaviside(a, b):
    return np.where(a==0, b, a > 0)
heaviside(np.arange(-5, 5), 0.3)

array([0. , 0. , 0. , 0. , 0. , 0.3, 1. , 1. , 1. , 1. ])

In [21]:
# Puzzle 20: repeat (1d)
# Goal: We have input of size (n,), and an integer d. We want to have output of size (d, n).
# Explanation: This can just be expressed as an outer product with an array of all 1s.
def repeat(a, d):
    return outer(ones(d), a)
repeat(np.arange(10), 5)

array([[0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
       [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
       [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
       [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
       [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]])

In [22]:
# Puzzle 21: Bucketize
# The definition asks us for the largest boundary with val >= boundaries[j]
# 
# Explanation: Matrix on LHS is X_{ij} = (values_i >= boundaries_j)
# Then if we sum over the rows of this, we get how many boundaries are at most the value, which is what we want.
# And we can sum over the rows by matrix producting with a vector of 1s.
def bucketize(values, boundaries): 
    return (values[:, None] >= boundaries) @ ones(boundaries.shape[0])
B = np.array([-3, 1, 4, 9])
A = np.arange(-6, 10)
vstack(A, bucketize(A, B))

array([[-6, -5, -4, -3, -2, -1,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9],
       [ 0,  0,  0,  1,  1,  1,  1,  2,  2,  2,  3,  3,  3,  3,  3,  4]])